# Chapter 7. Convolutional Neural Networks Applied to Image Classification

TENSORFLOW

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import logging
tf.get_logger().setLevel(logging.ERROR)

cifar = keras.datasets.cifar10 
(train_images, train_labels), (test_images, test_labels) = cifar.load_data()

print(f'Category: {train_labels[90]}')
plt.figure(figsize=(1, 1))
plt.imshow(train_images[90])
plt.show()

In [ ]:
## CONVOLUTIONAL NEURAL NETWORK
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPooling2D
import numpy as np 
import logging 
tf.get_logger().setLevel(logging.ERROR)

In [ ]:
EPOCHS = 128
BATCH_SIZE = 32

In [ ]:
# Load dataset
cifar_dataset = keras.datasets.cifar10 
(train_images, train_labels), (test_images, test_labels) = cifar_dataset.load_data()

In [ ]:
# Standardize dataset 
mean = np.mean(train_images)
stddev = np.std(train_images)
train_images = (train_images - mean) / stddev
test_images = (test_images - mean) / stddev

In [ ]:
# Change labels to one-hot.
train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

In [ ]:
# MODEL
model = Sequential()
model.add(Conv2D(64, (5,5), strides=(2,2), activation='relu', padding='same', input_shape=(32,32,3),
                 kernel_initializer='he_normal', bias_initializer='zeros'))
model.add(Conv2D(64, (3,3), strides=(2,2), activation='relu', padding='same', 
                 kernel_initializer="he_normal", bias_initializer='zeros'))
model.add(Flatten())
model.add(Dense(10, activation='softmax', 
                kernel_initializer='glorot_uniform', bias_initializer='zeros'))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(
    train_images, train_labels, validation_data=(test_images, test_labels),
    epochs=EPOCHS, batch_size=BATCH_SIZE,  verbose=2, shuffle=True
)

In [ ]:
# COUNTERING OVERFITTING
model = Sequential()
model.add(Conv2D(64, (4,4), activation='relu', padding='same', input_shape=(32,32,3)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (2,2), activation='relu', padding='same', strides=(2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_images, train_labels, validation_data=(test_images, test_labels),
                    epochs=EPOCHS, verbose=2, shuffle=True, batch_size=BATCH_SIZE)

PYTORCH

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import numpy as np
from utils import train_model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 128
BATCH_SIZE = 32

In [ ]:
# TRAINING DATASET
transform = T.Compose([T.ToTensor()])
trainset = CIFAR10(root='./pt_data', download=True, train=True, transform=transform)
testset = CIFAR10(root='./pt_data', train=False, download=True, transform=transform)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# MODEL
model = nn.Sequential(
    nn.Conv2d(3, 64, 4, stride=1, padding=1),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Conv2d(64, 64, 2, stride=2, padding=1),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Conv2d(64, 32, 3, stride=1, padding=1),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.MaxPool2d(2,2),
    nn.Flatten(),
    nn.Linear(32 * 8 * 8, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(64, 10)
)

In [ ]:
# Initialize weights with Xavier (Glorot) uniform for all weight layers.
for module in model.modules():
    if type(module) in {nn.Linear, nn.Conv2d}:
        nn.init.xavier_uniform_(module.weight)
        nn.init.constant_(module.bias, 0.0)

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
loss_function = nn.CrossEntropyLoss()

In [ ]:
# Train model.
train_model(model, device, EPOCHS, BATCH_SIZE, trainset, 
            testset, optimizer, loss_function, 'acc')